In [1]:
import codecs,unidecode,yaml,glob,re
import itertools as it
import pandas as pd

In [2]:
nQuizz=10
nAnnee=22
repQuizz="/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/"
fQuizzoodle=repQuizz+"QuizzAll.csv"
fAliasQDLE=repQuizz+"Quizz-Alias.yaml"
fNotesOriginales=repQuizz+"%d-Quizz-Notes.xlsx"%nAnnee
fNotesFinales=repQuizz+"%d-Quizz-Notes-Finales.xlsx"%nAnnee
fCM=repQuizz+"CM.csv"

In [3]:
'''https://stackoverflow.com/a/35491326/3986209'''
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = [key] 
            else: 
                inverse[item].append(key) 
    return inverse

# Fichier Etudiants

In [4]:
dfCM=pd.read_csv(filepath_or_buffer=fCM,sep="\t",encoding="utf8")
dfCM.set_index(u"Numéro d'identification",inplace=True)
dfCM[u"Code étudiant"]=dfCM.index.astype(unicode)
dfCM

,Nom,Prénom,Adresse de courriel,Dernier téléchargement depuis ce cours,Code étudiant
Numéro d'identification,,,,,
415923,AKKUL,EMMANUEL,emmanuel.akkul@etu.u-bordeaux-montaigne.fr,1664938189,415923
432822,AUBERT,EMMA,emma.aubert@etu.u-bordeaux-montaigne.fr,1664938189,432822
430711,AURIAULT,JUSTIN,justin.auriault@etu.u-bordeaux-montaigne.fr,1664938189,430711
431129,BARANGER,MANON,manon.baranger@etu.u-bordeaux-montaigne.fr,1664938189,431129
403123,BARSACQ,THEO,theo.barsacq@etu.u-bordeaux-montaigne.fr,1664938189,403123
433205,BEFORT,KRISTIN,kristin.befort@etu.u-bordeaux-montaigne.fr,1664938189,433205
428999,BENTO,ELENA,elena.bento@etu.u-bordeaux-montaigne.fr,1664938189,428999
429252,BERLOIN,AMANDINE,amandine.berloin@etu.u-bordeaux-montaigne.fr,1664938189,429252
419149,BERTRAND,SASCHA,sascha.bertrand@etu.u-bordeaux-montaigne.fr,1664938189,419149


# Fichier Alias

In [5]:
try:
    with codecs.open(fAliasQDLE, 'r', encoding="utf8") as inputfile:
        correspondances=yaml.safe_load(inputfile)
    alias=correspondances["alias2etu"]
except:
    alias={}
alias

{'ALINA': ['433401'],
 'Agathe': ['430440'],
 'Agathe / Liliana': ['430440', '430952'],
 'Agathe Martin': ['430440'],
 'Agathe martin': ['430440'],
 'Alex Helvert': ['380824'],
 'Alice LOOF': ['427886'],
 'Alima Gassama': ['418366'],
 'Alissa Bessettes': ['429623'],
 'Alissa bessettes': ['429623'],
 'Alix DEBUR': ['427449'],
 u'Ali\xe9nor Riou': ['428930'],
 'Amandine B': ['429252'],
 'Amandine Berloin': ['429252'],
 'Amandine berloin': ['429252'],
 'Amelie Mitlehner': ['433485'],
 u'Am\xe9lie rochefort': ['432360'],
 u'Ana\xeblle DINA-SAME': ['432151'],
 'Auriault justin': ['430711'],
 'Axel Meunier': ['429855'],
 'BENTO ELENA': ['428999'],
 'BENTO Elena': ['428999'],
 u'BOUSSARD L\xe9a': ['428452'],
 'Baptiste phelip': ['397583'],
 u'Barsacq Th\xe9o': ['403123'],
 'BarsacqTheo': ['403123'],
 'BarsacqqTheo': ['403123'],
 'Berloin Amandine': ['429252'],
 'Bertrand Sascha': ['419149'],
 'Blanc Sheryl': ['427234'],
 'Bonnet Nathan': ['433074'],
 u'Bordelais Cl\xe9a': ['429396'],
 u'Borde

In [6]:
colCM=[u"id",u"nom",u"prénom",u"email",u"ine"]
# colQuizz=["pseudo","points","temps"]

try:
    dfCM=pd.read_csv(filepath_or_buffer=fCM,sep="\t",encoding="utf8",header=0,names=colCM)
    lQuizz=pd.read_csv(filepath_or_buffer=fQuizzoodle,sep=";",encoding="utf8",header=0)
    lQuizz.columns=[c.strip() for c in lQuizz.columns]
    lQuizz.pseudo=lQuizz.pseudo.str.strip()
except:
    dfCM=None
    lQuizz=None

In [7]:
# pd.read_csv(filepath_or_buffer=fQuizzoodle,sep="\t",encoding="utf8",header=0)
dfCM

,id,nom,prénom,email,ine
0,415923,AKKUL,EMMANUEL,emmanuel.akkul@etu.u-bordeaux-montaigne.fr,1664938189
1,432822,AUBERT,EMMA,emma.aubert@etu.u-bordeaux-montaigne.fr,1664938189
2,430711,AURIAULT,JUSTIN,justin.auriault@etu.u-bordeaux-montaigne.fr,1664938189
3,431129,BARANGER,MANON,manon.baranger@etu.u-bordeaux-montaigne.fr,1664938189
4,403123,BARSACQ,THEO,theo.barsacq@etu.u-bordeaux-montaigne.fr,1664938189
5,433205,BEFORT,KRISTIN,kristin.befort@etu.u-bordeaux-montaigne.fr,1664938189
6,428999,BENTO,ELENA,elena.bento@etu.u-bordeaux-montaigne.fr,1664938189
7,429252,BERLOIN,AMANDINE,amandine.berloin@etu.u-bordeaux-montaigne.fr,1664938189
8,419149,BERTRAND,SASCHA,sascha.bertrand@etu.u-bordeaux-montaigne.fr,1664938189
9,429623,BESSETTES,ALISSA,alissa.bessettes@etu.u-bordeaux-montaigne.fr,1664938189


In [8]:
dfCM["PN"]=dfCM[u"prénom"].str.lower().str.strip()+" "+dfCM[u"nom"].str.lower().str.strip()
dfCM["NP"]=dfCM[u"nom"].str.lower().str.strip()+" "+dfCM[u"prénom"].str.lower().str.strip()

In [9]:
lQuizz

,pseudo,0(2),1(4),2(6),3(9),4(11),5(13),6(16),7(18),8(20),...,20(48),21(51),22(53),23(55),24(58),25(60),26(62),27(65),28(67),29(69)
0,,E,F,G,B,A,D,B,A,A,...,D,B,A,D,B,C,D,D,B,A
1,Manon Baranger,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,B,C,A
2,Mesquita Léo,E,F,G,D,B,B,D,?,B,...,B,A,A,D,B,C,D,C,D,A
3,Ugo /Agathe,?,D,G,D,?,C,A,?,?,...,?,?,?,?,?,?,?,?,?,?
4,Alissa bessettes,D,F,E,D,A,B,D,?,B,...,D,A,A,A,C,C,C,A,B,D
5,Amelie Mitlehner,E,F,G,D,C,D,B,A,A,...,B,B,A,C,C,C,B,B,?,B
6,Axel Meunier,D,F,?,B,A,D,B,A,B,...,D,C,A,C,B,C,D,B,B,A
7,Gaëlle Dubroca,E,F,D,D,B,B,D,E,A,...,D,B,A,D,B,C,D,C,D,A
8,Maewenn Godin,E,D,G,D,A,D,C,B,D,...,D,B,C,D,A,A,D,C,B,?
9,Monge clémence,E,D,F,B,A,D,A,B,B,...,D,C,A,D,C,C,D,B,?,A


In [10]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()
print "\n".join(pseudos)

In [11]:
for nP,pseudo in enumerate(pseudos):
    print pseudo
    sPseudo = raw_input("Nom a chercher:\n")
    if sPseudo:
        print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]
        idPseudo=raw_input("id :\n")
        if idPseudo:
            alias[pseudo]=idPseudo

Nom a chercher:
 


In [12]:
# pseudo=u"Cotentin, Pailhes"
# alias[pseudo]=["414180","411916"]

In [13]:
sPseudo="pailh"
print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]

Empty DataFrame
Columns: [PN, id]
Index: []


In [14]:
pn=dfCM["PN"].tolist()
np=dfCM["NP"].tolist()

In [15]:
for pseudo in pseudos:
    if unidecode.unidecode(pseudo.lower()) in pn:
        print pseudo,dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]
    elif unidecode.unidecode(pseudo.lower()) in np:
        print pseudo,dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]

In [16]:
for al in alias:
    if not isinstance(alias[al],list):
        alias[al]=[alias[al]]

In [17]:
invAlias=invert_dict(alias)
#invAlias[385732]=[u'adiarra001, cbeauvais004']
correspondances={"alias2etu":alias,"etu2alias":invAlias}
correspondances

{'alias2etu': {'ALINA': ['433401'],
  'Agathe': ['430440'],
  'Agathe / Liliana': ['430440', '430952'],
  'Agathe Martin': ['430440'],
  'Agathe martin': ['430440'],
  'Alex Helvert': ['380824'],
  'Alice LOOF': ['427886'],
  'Alima Gassama': ['418366'],
  'Alissa Bessettes': ['429623'],
  'Alissa bessettes': ['429623'],
  'Alix DEBUR': ['427449'],
  u'Ali\xe9nor Riou': ['428930'],
  'Amandine B': ['429252'],
  'Amandine Berloin': ['429252'],
  'Amandine berloin': ['429252'],
  'Amelie Mitlehner': ['433485'],
  u'Am\xe9lie rochefort': ['432360'],
  u'Ana\xeblle DINA-SAME': ['432151'],
  'Auriault justin': ['430711'],
  'Axel Meunier': ['429855'],
  'BENTO ELENA': ['428999'],
  'BENTO Elena': ['428999'],
  u'BOUSSARD L\xe9a': ['428452'],
  'Baptiste phelip': ['397583'],
  u'Barsacq Th\xe9o': ['403123'],
  'BarsacqTheo': ['403123'],
  'BarsacqqTheo': ['403123'],
  'Berloin Amandine': ['429252'],
  'Bertrand Sascha': ['419149'],
  'Blanc Sheryl': ['427234'],
  'Bonnet Nathan': ['433074'],

In [18]:
with codecs.open(fAliasQDLE, 'w', encoding="utf8") as outfile:
    yaml.safe_dump(correspondances, outfile, default_flow_style=False)

# Bonnes réponses pour les différents quizz du QuizzAll
- sur la deuxième ligne lQuizz[1], il y a toutes les bonnes réponses
- on coupe le Quizz All par groupe de trois questions pour chaque quizz d'origine
 - *quizzReponses*

In [19]:
with codecs.open(fQuizzoodle,"r",encoding="utf8") as input:
    lQuizz=input.readlines()

In [20]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]    

In [21]:
def getReponses(ligne):
    result={}
    ligneEls=[l.strip() for l in ligne.split(";")]
    pseudo=ligneEls[0]
    reponses=ligneEls[1:]
    for num,rep in enumerate(chunks(reponses,3)):
        result["%02d"%(num+2)]=rep
    return pseudo,result    

In [22]:
temp,quizzReponses=getReponses(lQuizz[1])
quizzReponses

{'02': [u'E', u'F', u'G'],
 '03': [u'B', u'A', u'D'],
 '04': [u'B', u'A', u'A'],
 '05': [u'B', u'C', u'D'],
 '06': [u'D', u'D', u'C'],
 '07': [u'B', u'A', u'C'],
 '08': [u'C', u'A', u'D'],
 '09': [u'B', u'A', u'D'],
 '10': [u'B', u'C', u'D'],
 '11': [u'D', u'B', u'A']}

In [23]:
def markReponses(reponses):
    scores={}
    for q in quizzReponses:
#         print q
        score=0
        for n,r in enumerate(quizzReponses[q]):
#             print n,r,reponses[q][n]
            if r==reponses[q][n]:
                score+=1
        scores[q]=int(round(float(score)*20/3,0))    
    return scores

# Notes du pseudo pour les différents quizz
- sur la ligne du pseudo, on a toutes les réponses
- on coupe les réponses par groupe de trois réponses
- on compare avec les bonnes réponses

In [24]:
alias[u"VSR"]

KeyError: u'VSR'

In [25]:
quizzNotes={}
for l in lQuizz[2:]:
    nom,reponses=getReponses(l)
#     print nom,reponses
    
    marks=markReponses(reponses)
    
#     print nom, " ; ".join([str(marks[k]) for k in sorted(marks.keys())])
    
    if nom in alias:
#         print nom,alias[nom]
        akas=alias[nom]
    else:
        print "pas de",nom
        akas=[]
        
    for aka in akas:
        if aka in quizzNotes:
            print aka,"déjà dans les notes",invAlias[aka]
            print quizzNotes[aka]
            print marks
            for k,v in quizzNotes[aka].iteritems():
                if v<marks[k]:
                    quizzNotes[aka][k]=marks[k]
            print quizzNotes[aka]
        else:
            quizzNotes[aka]=marks
print 
print "===================="
print
for aka in sorted(quizzNotes.keys()):
#    print chercherIdentifiant(aka)
    print aka, ", ".join([str(quizzNotes[aka][k]) for k in sorted(quizzNotes[aka].keys())])



380824 13, 20, 7, 13, 20, 7, 13, 13, 20, 13
382457 13, 0, 7, 13, 13, 13, 13, 20, 20, 7
385051 13, 7, 0, 20, 0, 7, 7, 0, 13, 0
403123 13, 20, 7, 7, 13, 0, 7, 0, 13, 7
416084 13, 0, 7, 20, 0, 13, 7, 0, 13, 7
417953 20, 20, 0, 7, 20, 7, 13, 13, 20, 20
419149 13, 13, 7, 7, 7, 13, 7, 7, 7, 7
424085 20, 7, 13, 13, 7, 7, 13, 0, 7, 13
425485 20, 13, 7, 7, 0, 0, 7, 13, 13, 7
426732 20, 0, 0, 13, 13, 13, 7, 13, 20, 7
426755 7, 20, 7, 20, 13, 7, 20, 20, 13, 0
426820 0, 13, 0, 7, 7, 7, 0, 0, 0, 0
426930 20, 20, 13, 13, 20, 7, 0, 20, 13, 7
427234 13, 20, 7, 13, 7, 7, 13, 20, 13, 7
427303 13, 13, 7, 13, 13, 0, 0, 20, 13, 0
427449 13, 20, 7, 20, 20, 13, 20, 13, 13, 13
427542 7, 13, 0, 13, 0, 7, 13, 13, 13, 0
427574 13, 13, 7, 20, 7, 7, 7, 13, 7, 7
427577 0, 20, 13, 20, 20, 13, 13, 13, 7, 13
427625 7, 20, 0, 13, 13, 7, 13, 20, 13, 0
427886 20, 13, 20, 20, 7, 13, 7, 7, 0, 0
427924 7, 7, 0, 0, 7, 13, 13, 7, 13, 13
427987 13, 13, 13, 0, 13, 13, 7, 13, 7, 7
428033 13, 13, 0, 13, 0, 7, 13, 13, 7, 7
42811

In [26]:
quizzNotes[u"433357"]

{'02': 20,
 '03': 0,
 '04': 13,
 '05': 20,
 '06': 0,
 '07': 7,
 '08': 13,
 '09': 0,
 '10': 0,
 '11': 7}

In [27]:
dfNotesOriginales=pd.read_excel(fNotesOriginales)
dfNotesOriginales[u"Numéro d'identification"]=dfNotesOriginales[u"Numéro d'identification"].astype("str")

In [31]:
# dfCampus=dfNotesOriginales[[u"Numéro d'identification"]+u"Nom Prénom".split(" ")+["Quizz%02d (Brut)"%(i+1) for i in range(1,10)]]
dfCampus=dfNotesOriginales[[u"Numéro d'identification"]+u"Nom Prénom".split(" ")+["Q%02d"%(i+1) for i in range(1,11)]]
dfCampus.set_index(u"Numéro d'identification",inplace=True)
# dfCampus.loc["417001"]

In [32]:
nouvellesNotes={}
for etu in quizzNotes:
    print etu
    if etu in dfCampus.index:
        for q in quizzNotes[etu]:
            # if quizzNotes[etu][q] > dfCampus.loc[etu,"Quizz%s (Brut)"%q] or dfCampus.loc[etu,"Quizz%s (Brut)"%q]=="-":
            if quizzNotes[etu][q] > dfCampus.loc[etu,"Q%s"%q] or dfCampus.loc[etu,"Q%s"%q]=="-":
                # print u"Quizz %s"%q,dfCampus.loc[etu,"Quizz%s (Brut)"%q],"=>",quizzNotes[etu][q]
                print u"Quizz %s"%q,dfCampus.loc[etu,"Q%s"%q],"=>",quizzNotes[etu][q]
                if not etu in nouvellesNotes:
                    nouvellesNotes[etu]={}
                nouvellesNotes[etu][q]=quizzNotes[etu][q]
                # dfCampus.loc[etu,"Quizz%s (Brut)"%q]=quizzNotes[etu][q]
                dfCampus.loc[etu,"Q%s"%q]=quizzNotes[etu][q]
    else:
        print "PB", etu, "pas de notes originales"
        print quizzNotes[etu]
        nouvellesNotes[etu]=quizzNotes[etu]
len(nouvellesNotes)

433618
Quizz 06 - => 7
Quizz 11 - => 0
428617
Quizz 03 16 => 20
Quizz 10 - => 13
432779
Quizz 03 - => 13
Quizz 06 11 => 13
Quizz 07 - => 0
Quizz 05 11 => 13
Quizz 09 - => 20
Quizz 11 12 => 13
Quizz 10 - => 20
433135
Quizz 10 12 => 13
427234
Quizz 02 11 => 13
Quizz 03 16 => 20
Quizz 05 10 => 13
Quizz 08 10 => 13
Quizz 09 - => 20
Quizz 10 - => 13
424085
Quizz 02 16 => 20
429928
Quizz 11 12 => 13
431463
428452
Quizz 02 13 => 20
Quizz 07 10 => 13
433322
Quizz 02 - => 13
Quizz 03 14 => 20
Quizz 11 - => 7
382457
Quizz 09 17 => 20
Quizz 10 18 => 20
431197
Quizz 07 - => 7
Quizz 05 17 => 20
Quizz 08 15 => 20
Quizz 11 12 => 20
428197
Quizz 03 11 => 20
Quizz 06 13 => 20
Quizz 09 - => 7
Quizz 10 12 => 13
429273
Quizz 06 11 => 13
425485
Quizz 02 14 => 20
Quizz 03 - => 13
Quizz 06 - => 0
Quizz 11 - => 7
427987
431735
Quizz 02 14 => 20
Quizz 09 - => 7
Quizz 11 - => 7
Quizz 10 - => 7
385051
Quizz 05 16 => 20
Quizz 10 12 => 13
429499
Quizz 03 - => 13
Quizz 04 17 => 20
Quizz 10 13 => 20
380824
Quizz 03 

58

In [34]:
dfCampus.loc["427303"]

Nom       DELAHODDE
Prénom          EVA
Q02              14
Q03              16
Q04              11
Q05              13
Q06              13
Q07              13
Q08              13
Q09              20
Q10              13
Q11              13
Name: 427303, dtype: object

In [35]:
numId="427303"
dfCampus.loc[numId],dfNotesOriginales[dfNotesOriginales[u"Numéro d'identification"]==numId]

(Nom       DELAHODDE
 Prénom          EVA
 Q02              14
 Q03              16
 Q04              11
 Q05              13
 Q06              13
 Q07              13
 Q08              13
 Q09              20
 Q10              13
 Q11              13
 Name: 427303, dtype: object,
    Numéro d'identification        Nom Prénom Q02 Q03 Q04 Q05 Q06 Q07 Q08 Q09  \
 39                  427303  DELAHODDE    EVA  14  16  11  11   -  13  13  13   
 
    Q10 Q11  
 39   -  13  )

In [36]:
dfCampus.to_excel(fNotesFinales)